### Load packages and sample data

In [1]:
import shlex
import re
import pandas as pd
import numpy as np
from typing import List
import utils

In [2]:
fields_to_include = ["OWNER","TITLE", "ABSTRACT", "SERVICETYPE", "SERVICELINK", "KEYWORDS"]
fields_to_output = ["OWNER","TITLE", "SERVICETYPE", "SERVICELINK"]
url_github_repo = "https://github.com/davidoesch/geoservice_harvester_poc/blob/main/data/"
url_github_repo_suffix = "?raw=true"
url_geoservices_CH_csv = "{}geoservices_CH.csv{}".format(url_github_repo,url_github_repo_suffix)
a = 'Strassen auswertung system'

In [3]:
def split_delimiters(word_list_with_delimiters: List[str]) -> List[str]:
    """Take care of left over delimiters, split strings even if in qoutes
        Return a list of words """
    delimiters = [";", ","]

    new_word_list = []

    for word in word_list_with_delimiters:
        if (any(delimiter in word for delimiter in delimiters)):
            splitted_words = re.split(r',|;', word)
            for splitted_word_ in splitted_words:
                new_word_list.append(splitted_word_)
        else:
            new_word_list.append(word)
    return new_word_list

def load_data():
    dataframe = pd.read_csv(url_geoservices_CH_csv, usecols=fields_to_include)
    return dataframe

In [4]:
res = split_delimiters(shlex.split(a))
dataframe_some_cols = load_data()
result = dataframe_some_cols[dataframe_some_cols.apply(lambda dataset: dataset.astype(str).str.contains(res[0], case=False).any(), axis=1)]
result =result.fillna("empty")

### TF-IDF ranking of the abstract

In [9]:
result['tfidf'] = result.apply(lambda row: utils.ranking_tfidf(row['ABSTRACT']), axis=1)


In [19]:
result.head()

,OWNER,TITLE,ABSTRACT,KEYWORDS,SERVICELINK,SERVICETYPE,tfidf
0,KT_AI,Abwasser Werkplan Gde,Diese Karte zeigt die Werkleitungen der Abwass...,empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(sowi, 0.54), (abwasserentsorgung, 0.5), (ka..."
65,KT_AI,Hanglagen CH,"Geodaten des Bundesamtes für Strassen, als WMS...",empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(hangneigungsklassen, 0.88), (datensatz, 0.8..."
66,KT_AI,Hauptstrassennetz CH,"Geodaten des Bundesamtes für Strassen, als WMS...",empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(strassen, 0.78), (wm, 0.7), (hauptstrassen,..."
68,KT_AI,Höhenkurven Kt,Diese Karte beinhaltet die Höhenkurven mit ein...,empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(höhenkurven, 0.84), (auswertung, 0.47), (be..."
70,KT_AI,IVS Geländekarte CH,"Geodaten des Bundesamtes für Strassen, als WMS...",empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(viv, 0.85), (geführt, 0.77), (wm, 0.71), (k..."


In [16]:
id = 70
result.loc[id]['ABSTRACT'], result.loc[id]['tfidf']

('Geodaten des Bundesamtes für Strassen, als WMS der swisstopo bezogen. Die Geländekarte stellt die kartografische Grundlage der Ersterfassung des Inventars dar. Sie beinhaltet ergänzende Angaben zu den Objekten nach Artikel 3 der Verordnung über das Bundesinventar der historischen Verkehrswege der Schweiz VIVS sowie zu den Objekten von regionaler und lokaler Bedeutung. Die Geländekarte wird vorerst nicht nachgeführt und deshalb nur als Zusatzinformation in der elektronischen Publikation sowie als gesonderte Karte in der gedruckten Ausgabe (vgl. Art. 4 VIVS) geführt. Der Kanton Solothurn hat für die Neubeurteilung der Objekte von regionaler/lokaler Bedeutung die Geländekarten überarbeitet. Diese überarbeiteten Versionen sind im vorliegenden Datensatz enthalten. WMS Service Geoportal - Kanton Appenzell Innerrhoden',
 ([('viv', 0.85),
   ('geführt', 0.77),
   ('wm', 0.71),
   ('kanton', 0.65),
   ('geländekart', 0.55),
   ('bedeutung', 0.5),
   ('lokal', 0.5),
   ('objekten', 0.49),
   (

In [17]:
result.loc[id]['ABSTRACT'], result.loc[id]['tfidf'], result.loc[id]['KEYWORDS'], result.loc[id]['TITLE']

('Geodaten des Bundesamtes für Strassen, als WMS der swisstopo bezogen. Die Geländekarte stellt die kartografische Grundlage der Ersterfassung des Inventars dar. Sie beinhaltet ergänzende Angaben zu den Objekten nach Artikel 3 der Verordnung über das Bundesinventar der historischen Verkehrswege der Schweiz VIVS sowie zu den Objekten von regionaler und lokaler Bedeutung. Die Geländekarte wird vorerst nicht nachgeführt und deshalb nur als Zusatzinformation in der elektronischen Publikation sowie als gesonderte Karte in der gedruckten Ausgabe (vgl. Art. 4 VIVS) geführt. Der Kanton Solothurn hat für die Neubeurteilung der Objekte von regionaler/lokaler Bedeutung die Geländekarten überarbeitet. Diese überarbeiteten Versionen sind im vorliegenden Datensatz enthalten. WMS Service Geoportal - Kanton Appenzell Innerrhoden',
 ([('viv', 0.85),
   ('geführt', 0.77),
   ('wm', 0.71),
   ('kanton', 0.65),
   ('geländekart', 0.55),
   ('bedeutung', 0.5),
   ('lokal', 0.5),
   ('objekten', 0.49),
   (

### Improvement of the TF-IDF with BM25 to execute a query on the text

In [6]:
# from sklearn.datasets import fetch_20newsgroups
# texts = fetch_20newsgroups(subset='train').data
abstracts = result['ABSTRACT'].values.tolist()

In [9]:
bm25 = utils.TFIDF_BM25()
bm25.fit(abstracts[1:])

"Die Karte zeigt, welcher Lärmbelastung die Bevölkerung durch den Strassenverkehr in der Nacht ausgesetzt ist. Die Angaben basieren auf flächendeckenden Modellberechnungen (sonBASE). Das gesamte Strassennetz der Schweiz, das in diese Berechnung einbezogen wurde, umfasst ca. 68'000 km. Die Verkehrsdaten für den Strassenverkehr wurden mit einem Mobilitätsmodell für das Jahr 2015 ermittelt. Die so ermittelten Verkehrsdaten wurden auf der Grundlage von ca. 1900 nationalen und kantonalen Zählstellen mit stundenfeinen Zählwerten kalibriert. Daten sind gesetzlich nicht verbindlich. Verbindliche Angaben zur Belastung wie auch zur Lärmsanierung geben die jeweiligen Vollzugsbehörden. Bei Nationalstrassen: Bundesamt für Strassen (ASTRA). Bei Haupt- und übrige Strassen: Die Kantonalen Vollzugsbehörden. WMS Service Geoportal - Kanton Appenzell Innerrhoden"

In [15]:
res = bm25.transform('mit', abstracts)

In [16]:
np.where(res > 0)[0].tolist()

[0,
 3,
 5,
 6,
 10,
 11,
 12,
 13,
 14,
 16,
 19,
 21,
 22,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 54,
 55,
 56,
 57,
 63,
 65,
 76,
 77,
 79,
 209,
 214,
 215,
 216,
 217,
 231,
 232,
 348,
 349,
 350,
 351,
 352,
 353,
 358,
 359,
 360,
 361,
 362,
 363,
 396,
 397,
 398,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 429,
 430,
 431,
 436,
 438,
 440,
 441,
 442,
 443,
 444,
 445,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 453,
 454,
 455,
 456,
 457,
 458,
 460,
 461,
 462,
 463,
 466,
 471,
 472,
 491,
 492,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 523,
 524,
 525,
 529,
 530,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 543,
 544,
 547,
 555,
 556,
 557,
 558,
 559,
 561,
 562,
 563,
 564,
 565,
 575,
 576,
 577,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 588,
 589,
 593,
 614,
 641,
 642,
 656,
 657,
 658,


In [14]:
abstracts[54]

'Der Datensatz "Durchlass" bildet zusammen mit den Datensätzen "Eimer", "Fotopunkt", "Leitsystem" und "Stopprinne" eine Präzisierung zum Datensatz "Amphibienzugstellen". Er enthält die Durchlässe der Amphibienzugstellen des Kantons Aargau. Dies sind Bauten, welche den Amphibien ermöglichen, eine Strasse unterirdisch zu passieren. Ihr Standort wurde im Feld durch die Aufnahme eines Punktes auf mindestens einer Strassenseite ermittelt (Aufnahmegerät: GPS von der Firma Leica, System 1200, Genauigkeit im Wald: 1m). Gestützt auf diese Punkte wurden im Büro die Durchlässe als Polylinie digitalisiert. Aus dem Datensatz atb_kuba wurden als Linie Objekte gemäss Auswahl von Thomas Gerber übernommen. Das Attribut CK aus atb_kuba wurde in das Attribut kuba_ck übernommen. In der Bemerkung steht, aus welchem Zeitstand atb_kuba das Objekt übernommen wurde. Der Datensatz "Durchlass" bildet zusammen mit den Datensätzen "Eimer", "Fotopunkt", "Leitsystem" und "Stopprinne" eine Präzisierung zum Datensatz 